In [1]:
import time
import os.path
from utils import *
from pymoo.operators.sampling.lhs import LHS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymoo.core.problem import Problem
from pflacco.classical_ela_features import *
from pflacco.misc_features import *

from pymoo.algorithms.soo.nonconvex.de import DE
from pymoo.algorithms.soo.nonconvex.pso import PSO
from pymoo.operators.sampling.lhs import LHS
from pymoo.optimize import minimize
from pymoo.termination import get_termination
from pymoo.algorithms.soo.nonconvex.cmaes import CMAES
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.algorithms.soo.nonconvex.es import ES
from pymoo.core.callback import Callback

In [2]:
def algorithm_factory(name, population=20):
    if name == 'GA':
        return GA(pop_size=population, sampling=LHS())
    elif name == 'PSO':
        return PSO(pop_size=population, sampling=LHS())
    elif name == 'DE':
        return DE(pop_size=population, sampling=LHS(), CR=0.1)
    elif name == 'CMAES':
        return CMAES(popsize=population)
    elif name == 'ES':
        return ES(pop_size=population)
    else:
        raise Exception("No algorithm")
        
def run_algorithms(problem, n_runs=10, n_eval=1000, algorithms = ['GA', 'PSO', 'DE', 'CMAES', 'ES']):
    algo_data = {x: [] for x in algorithms}
    algo_data['algorithm_run'] = []
    for run in range(n_runs):
        
        algo_data['algorithm_run'].append(run)
        for algo_name in algorithms:
            algorithm = algorithm_factory(algo_name)
            termination=get_termination("n_eval", n_eval)
            res = minimize(problem, algorithm,
                           termination=termination,
                           verbose=False, 
                           save_history=False)
            algo_data[algo_name].append(res.F[0])
    return algo_data

def get_rank(full_rank):
    all_algos = []
    for x in full_rank:
        all_algos.extend(x)
    all_algos = list(set(all_algos))
    rd = {}
    for a in all_algos:
        si = 0
        for r in full_rank:
            i = r.index(a)
            si += i
        si /= len(full_rank)
        rd[a] = si
    return rd

In [3]:
if __name__ == "__main__":
    dim = 2
    n_runs=1
    n_eval_dim_multip = 1000
    directory = 'mix_problems_runs'
    create_directory_if_not_exist(directory)
    problem_instance = 1
    for problem_class1 in range(1, 25):
        for problem_class2 in range(1, problem_class1):
            problem1 = COCO(problem_class1, 1, 2)
            problem2 = COCO(problem_class2, 1, 2)
            for mix in [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]:
                print(problem_class1, problem_class2, mix)
                
                
                file = f'{directory}/dim_{dim}_problem1_{problem_class1}_problem2_{problem_class2}_mix_{mix}_instance_{problem_instance}_budgetdim_{n_eval_dim_multip}.csv'
                if os.path.exists(file):
                    print(f"Skip {file}")
                    continue

                start_time = time.time()
                problem = ProblemMix(problem1, problem2, mix=mix)
                runs = run_algorithms(problem, n_runs=n_runs, n_eval=dim*n_eval_dim_multip)
                runs['problem_class'] = n_runs*[problem_class]
                runs['problem_instance'] = n_runs*[problem_instance]
                print(runs)
                print("--- %s seconds ---" % (time.time() - start_time))
                df = pd.DataFrame(runs)
                df.to_csv(file, index=False)

2 1 0.0
(1, 2)
[27562422.5795003]
[121.84209408]
27555306.82377357
62.59202806078808
adasd [1.00026581]
dfgdgd [0.67675735]
adfadfafaf [1.00026581]
(20, 2)
[27562422.5795003 27562422.5795003 27562422.5795003 27562422.5795003
 27562422.5795003 27562422.5795003 27562422.5795003 27562422.5795003
 27562422.5795003 27562422.5795003 27562422.5795003 27562422.5795003
 27562422.5795003 27562422.5795003 27562422.5795003 27562422.5795003
 27562422.5795003 27562422.5795003 27562422.5795003 27562422.5795003]
[121.84209408 121.84209408 121.84209408 121.84209408 121.84209408
 121.84209408 121.84209408 121.84209408 121.84209408 121.84209408
 121.84209408 121.84209408 121.84209408 121.84209408 121.84209408
 121.84209408 121.84209408 121.84209408 121.84209408 121.84209408]
27555306.82377357
62.59202806078808
adasd [1.00026581 1.00026581 1.00026581 1.00026581 1.00026581 1.00026581
 1.00026581 1.00026581 1.00026581 1.00026581 1.00026581 1.00026581
 1.00026581 1.00026581 1.00026581 1.00026581 1.00026581 1

/opt/conda/envs/Base/lib/python3.10/site-packages/pymoo/util/misc.py:99: RuntimeWarning: invalid value encountered in true_divide
  return lambda a, b: np.sqrt((((a - b) / (xu - xl)) ** 2).sum(axis=1))


TypeError: '>' not supported between instances of 'NoneType' and 'NoneType'